In [1]:
import os

import torch
import torch.optim as optim
import torch.nn as nn

from datasets import load_dataset
from trl import GRPOTrainer
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset

device = torch.device("mps")

In [2]:
dataset = load_dataset("trl-lib/tldr", split="train")
print(dataset)

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 116722
})


In [3]:
train_dataset = Dataset.from_dict(dataset[:116000])
print(train_dataset)

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 116000
})


In [4]:
val_dataset = Dataset.from_dict(dataset[116000:116300])
test_dataset = Dataset.from_dict(dataset[116300:])

In [5]:
MODEL_NAME = "Qwen/Qwen3-0.6B"

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map=device)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [6]:
print(model)

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [6]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    task_type=TaskType.CAUSAL_LM,
    target_modules=[
        "model.layers.26.mlp.gate_proj",
        "model.layers.26.mlp.up_proj",
        "model.layers.26.mlp.down_proj"
    ]
)
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 196,608 || all params: 596,246,528 || trainable%: 0.0330


In [7]:
print(model)

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-25): 26 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [38]:
_idx = 2

prompt = test_dataset[_idx]["prompt"]
completion = test_dataset[_idx]["completion"]

print(f"Prompt: {prompt}")
print("-"*10)
print(f"Ground truth: {completion}")
print("-"*10)

inputs = tokenizer.encode(prompt, return_tensors="pt").to("mps")
prompt_length = inputs.shape[1]
print(f"{inputs.shape=}")

output = model.generate(inputs, max_new_tokens=100)
print(f"Output: {tokenizer.decode(output[0, prompt_length:])}")

Prompt: SUBREDDIT: r/relationships

TITLE: I [18 M] don't know how to avoid with a controlling man [30's M] who want's to be friends.

POST: Yeah, reaaaal creepy. I think he may be gay, I dunno. He's not put a move on me or anything, but still.

This guy's not respecting my space, everytime we meet and talk he always has some complicated load to dump on me. I hate it! I just feel his negative selfish energy sapping my own away. I never leave him happy, EVER! It's a classic therapist/patient relationship. The guy just needs to shut his mouth.

So, I tried cutting ties with him, but this guy is a bit cuckoo and doesn't have any other friends/family to go to. So, he is clinging like a mofo. I told him explicitly that I want to be alone for a week or two and he totally blew me off saying I'm selfish and not a good friend. I swear he's a bit of a narcissist or a controlling personallity or both.

The worst part though, is since we are both religious, he pulled the "what would Jesus do" card

### Generate samples

In [46]:
_idx = 10
batch_size = 8
max_new_tokens = 200

prompt = train_dataset[_idx]["prompt"]
completion = train_dataset[_idx]["completion"]

inputs = tokenizer.encode(prompt, return_tensors="pt").to("mps")
inputs = inputs.expand(batch_size, -1)
prompt_length = inputs.shape[1]
print(f"{inputs.shape=}")

print("-"*50)
print(f"Prompt: {prompt}")
print("-"*5)
print(f"Ground truth: {completion}")
print("-"*5)

output = model.generate(inputs, max_new_tokens=100)
print(f"Output: {tokenizer.decode(output[0, prompt_length:])}")

inputs.shape=torch.Size([8, 255])
--------------------------------------------------
Prompt: SUBREDDIT: r/offmychest

TITLE: I'm just really tired right now...

POST: I just discovered this subreddit and I am glad I did because I am about to be at the breaking point.
I am about to move out of my house with my boyfriend and into an apartment in about a week. I've been extremely stressed out. I am losing sleep about this and I have been becoming irritated at many things. Having enough money has been something that has haunted me most of all. I am worried that we won't be able to make it. I worry about how we will be living and if we will even have food at the table. I have talked to a couple of friends about this and they have all told me that if I'm not ready to move out then I shouldn't, but I have lived a very abusive life here at my house so moving is for the best. I'm probably just scared of the unknown. My boyfriend doesn't seem to be worried at all about moving out and it ticks me

In [50]:
texts = tokenizer.batch_decode(output[:,prompt_length:])

In [52]:
for _i, each in enumerate(texts):
    print("-"*5 + f" {_i}")
    print(each)

----- 0
 I'm tired, scared, overthinking, and anxious about moving out.

This is the first time I've ever done this, so I can't even put this into a positive context. I can only say that it's a good thing to do. I just want to move out. I don't know why I'm doing this.

The post is written in a very emotional tone. It's very personal and I think it's a good example of how to express a personal issue. It's
----- 1
 I'm tired, worried, and over thinking about moving out.

Okay, so the user is talking about moving out of their house with their boyfriend, which they're planning to do in about a week. They're extremely stressed and have been losing sleep. They're worried about money, food, and the future. They've talked to friends, but they're not sure if moving is for them. They're scared of the unknown and their boyfriend isn't paying attention to their situation. They might be
----- 2
 I'm tired, stressed, worried, and I have a lot of questions but am not ready to move out.

Okay, I need

In [8]:
import re

def remove_think_tags(text):
    return re.sub(r'<think>[\s\S]*?</think>', '', text)

In [166]:
_in = tokenizer.apply_chat_template(
        [{"role": "user", "content": f"/no_think Summarize this post very short and concise {prompt}"}], add_generation_prompt=True, return_tensors="pt"
).to("mps")

_out = model.generate(
    _in,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.9
)

print(tokenizer.decode(_in[0]))
print("Completion " + "-"*5)
print(completion)
print("Output " + "-"*5)
generated = tokenizer.decode(_out[0, _in.shape[1]:])
generated = remove_think_tags(generated).strip().replace("<|im_end|>", "")
print(generated)

<|im_start|>user
/no_think Summarize this post very short and concise SUBREDDIT: r/offmychest

TITLE: I'm just really tired right now...

POST: I just discovered this subreddit and I am glad I did because I am about to be at the breaking point.
I am about to move out of my house with my boyfriend and into an apartment in about a week. I've been extremely stressed out. I am losing sleep about this and I have been becoming irritated at many things. Having enough money has been something that has haunted me most of all. I am worried that we won't be able to make it. I worry about how we will be living and if we will even have food at the table. I have talked to a couple of friends about this and they have all told me that if I'm not ready to move out then I shouldn't, but I have lived a very abusive life here at my house so moving is for the best. I'm probably just scared of the unknown. My boyfriend doesn't seem to be worried at all about moving out and it ticks me off because it feels l

In [9]:
import re

def get_final_content(text):
    return re.findall(r'<final>([\s\S]*?)</final>', text)

In [170]:
_out2 = model.generate(
    tokenizer.apply_chat_template(
        [
            {"role": "user", "content": f"How close in meaning is the 1st sentence to the 2nd sentence, from 0 (none) to 5 (identical). Put the number in <final></final> tag:\nSentence 1: {generated}\nSentence 2: {completion}"}
        ], add_generation_prompt=True, return_tensors="pt"
    ).to("mps"),
    max_new_tokens=2048,
    do_sample=True,
    temperature=0.9
)
print(tokenizer.decode(_out2[0]))

<|im_start|>user
How close in meaning is the 1st sentence to the 2nd sentence, from 0 (none) to 5 (identical). Put the number in <final></final> tag:
Sentence 1: r/offmychest  
TL;DR: I'm tired and scared of moving out—my boyfriend's not worried.
Sentence 2:  I'm worried about moving out of parent's house and into an apartment with a boyfriend who doesn't really seemed worried at all.<|im_end|>
<|im_start|>assistant
<think>
Okay, let's see. The user is asking how close in meaning the two sentences are, from 0 to 5. First, I need to compare the two sentences.

Starting with the first sentence: "r/offmychest". That seems like a typo or maybe a misspelling. Maybe "r/offmychest" was intended to be "r/ my chest" but with some other text. But regardless, the user provided that as part of the input, so I should focus on the meaning here.

Now, the second sentence: "I'm worried about moving out of parent's house and into an apartment with a boyfriend who doesn't really seemed worried at all." 

In [125]:
print(tokenizer.decode(_out2[0]))

<|im_start|>user
/no_think How close in meaning is the 1st sentence to the 2nd sentence, from 0 (none) to 5 (identical). Put the number in <final></final> tag:
Sentence 1: r/offmychest  
I'm just tired now. I moved out of my house with my boyfriend soon, and I'm really stressed about the future.<|im_end|>
Sentence 2:  I'm worried about moving out of parent's house and into an apartment with a boyfriend who doesn't really seemed worried at all.<|im_end|>
<|im_start|>assistant
<think>

</think>

<final>3</final><|im_end|>


### Reward functions

In [10]:
import re

def get_final_content(text):
    return re.findall(r'<final>([\s\S]*?)</final>', text)
    # text = re.sub(r'<think>[\s\S]*?</think>', '', text)
    # return re.findall(r'-?\d+\.?\d*', text)

@torch.no_grad()
def reward_meaning(prompt, generated, ground_truth, n_calls=3, _verbose=False):
    token_ids = tokenizer.apply_chat_template(
        [
            {"role": "user", "content": (
                f"Original text: {prompt}\n\n"
                f"\nSummary 1: {generated}\nSummary 2: {ground_truth}\n\n"
                f"How close in meaning is the 1st summary to the 2nd summary, "
                f"from 0 (none) to 5 (identical). Put the final score within <final></final> tag.")
            }
        ], add_generation_prompt=True, return_tensors="pt",
    ).to("mps")
    token_ids = token_ids.expand(n_calls, -1)
    with peft_model.disable_adapter():
        _out2 = peft_model.generate(
            token_ids,
            max_new_tokens=2048,
            do_sample=True,
            temperature=0.9
        )
    scores, score_debugs = [], []
    for n_call in range(n_calls):
        out_text = tokenizer.decode(_out2[n_call, token_ids.shape[1]:])
        score_debugs.append(out_text)
        matched_final_content = get_final_content(out_text)
        if _verbose:
            print("-"*10)
            print(out_text)
            print(matched_final_content)
        if not matched_final_content:
            continue
        for matched_content in reversed(matched_final_content):
            try:
                scores.append(float(matched_content))
                break
            except Exception as e:
                print(e)
    if scores:
        return sum(scores) / len(scores), score_debugs
    return 0, score_debugs

### Training behavior

In [11]:
import time

In [11]:
peft_model.add_adapter("ref_model", peft_config=peft_config, low_cpu_mem_usage=True)

In [11]:
print(peft_model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 1024)
        (layers): ModuleList(
          (0-25): 26 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
              (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
              (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
              (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
              (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
            )
            (mlp): Qwen3MLP(
              (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
              (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
              (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          

In [14]:
with torch.no_grad():
    print(peft_model.base_model.model.model.layers[26].mlp.gate_proj.lora_A.default.weight.sum())
    print(peft_model.base_model.model.model.layers[26].mlp.gate_proj.lora_A.default.weight.requires_grad)
    print(peft_model.base_model.model.model.layers[26].mlp.gate_proj.lora_A.ref_model.weight.sum())
    print(peft_model.base_model.model.model.layers[26].mlp.gate_proj.lora_A.ref_model.weight.requires_grad)

tensor(1.5121, device='mps:0')
True
tensor(..., device='meta', size=())
False


In [12]:
def copy_weight(copy_from, copy_to, peft_model):
    copy_from = "default"
    copy_to = "ref_model"
    for _name, _mod in peft_model.named_modules():
        if not isinstance(_mod, nn.ModuleDict):
            continue
        if copy_from not in _mod:
            # print(f"{_name}: No {copy_from}")
            continue
        if copy_to not in _mod:
            # print(f"{_name}: No {copy_to}")
            continue
        # print(f"Copying from {copy_from} to {copy_to}")
        _mod_from = _mod[copy_from]
        _mod_to = _mod[copy_to]
    
        if not isinstance(_mod_from, _mod_to.__class__):
            print(f"{_name}: Not same type {type(_mod_from)=}, {type(_mod_to)=}")
            continue
        
        for _pname, _from_param in _mod_from.named_parameters():
            _new_to_param = nn.Parameter(
                data=torch.clone(_from_param.data),
                requires_grad=getattr(_mod_to, _pname).requires_grad,
            )
            delattr(_mod_to, _pname)
            _mod_to.register_parameter(_pname, _new_to_param)
    
        for _bname, _from_buffer in _mod_from.named_buffers():
            _to_buffer = _mod_to.get_buffer(_bname)
            _to_buffer.data = torch.clone(_from_buffer.data)

In [401]:
peft_model.train(True)
peft_model.training

True

In [16]:
for _name, _mod in peft_model.named_modules():
    for _k in _mod._forward_pre_hooks.keys():
        _mod._forward_pre_hooks.pop(_k)

In [19]:
with torch.no_grad():
    trained_params = [torch.clone(p) for p in peft_model.parameters() if p.requires_grad]

In [13]:
_rq_params = [p for p in peft_model.parameters() if p.requires_grad]
optimizer = optim.Adam(params=_rq_params, lr=1e-4)

In [14]:
with torch.no_grad():
    original_params = [torch.clone(p) for p in peft_model.parameters() if p.requires_grad]

In [18]:
batch_size = 3
max_new_tokens = 200
sync_ref_freq = 100    # synchronize the reference model every `sync_ref_freq` iterations
mu = 3
epsilon = 0.2
beta = 1e-3
_verbose = True

_eos_id_check = tokenizer.encode("<|endoftext|>")
if len(_eos_id_check) != 1:
    raise ValueError("Supply the correct end-of-sequence token for this model")
eos_id = _eos_id_check[0]

for _i, _data in enumerate(train_dataset):
    # if _i < 40:
    #     continue
    prompt = _data["prompt"]
    completion_ground_truth = _data["completion"]
    inputs = tokenizer.apply_chat_template(
        [{"role": "user", "content": f"/no_think Summarize this post very short and concise {prompt}"}], 
        add_generation_prompt=True, return_tensors="pt"
    ).to("mps")
    inputs = inputs.expand(batch_size, -1)
    prompt_length = inputs.shape[1]

    # generate the data
    with torch.no_grad():
        data_output = peft_model.generate(inputs, max_new_tokens=100)
        generated_candidates = tokenizer.batch_decode(data_output[:,prompt_length:])
        generated_candidates = [
            remove_think_tags(generated).strip().replace("<|im_end|>", "").replace("<|endoftext|>", "")
            for generated in generated_candidates
        ]
        ground_truth = data_output[:, inputs.shape[1]:]
        mask = ground_truth != eos_id


    if _verbose:
        print("-"*50)
        print(prompt)
        print("-"*10)
        print(completion_ground_truth)
        print("-"*10)

    # get the reference output
    if (_i != 0) and (_i % sync_ref_freq == 0):
        copy_weight("default", "ref_model", peft_model)
    with torch.no_grad():
        peft_model.train(False)     # weird require from `peft` to disable training in order to supply `adapter_names`
        if _i <= sync_ref_freq:     # haven't synchronized the ref model
            with peft_model.disable_adapter():
                ref_logits = peft_model(data_output).logits[:, inputs.shape[1]-1:-1]
        else:
            ref_logits = peft_model(data_output, adapter_names=["ref_model"] * batch_size).logits[:, inputs.shape[1]-1:-1]
        ref_logsumexp = torch.logsumexp(ref_logits, dim=-1, keepdim=True)
        ref_logprobs = ref_logits - ref_logsumexp
        ref_logps = torch.gather(ref_logprobs, dim=-1, index=ground_truth[...,None]).squeeze() * mask
        peft_model.train(True)

    # score the rewards
    meaning_scores = []
    for _candidate in generated_candidates:
        _score, _score_debug = reward_meaning(prompt, _candidate, completion_ground_truth, n_calls=2)
        meaning_scores.append(_score)
        if _verbose:
            print(_score, _candidate)
            print("-"*2)

    if _verbose:
        print("-"*10)
        with peft_model.disable_adapter(), torch.no_grad():
            data_output2 = peft_model.generate(inputs, max_new_tokens=100)
            generated_candidates2 = tokenizer.batch_decode(data_output2[:,prompt_length:])
            generated_candidates2 = [remove_think_tags(generated).strip().replace("<|im_end|>", "").replace("<|endoftext|>", "") for generated in generated_candidates2]
            print("Without peft:")
            for __each in generated_candidates2:
                print(__each)
                print("-")

    # get the advantages
    meaning_scores = torch.tensor(meaning_scores)
    if meaning_scores.sum() == 0:
        if os.environ.get("EXIT_PDB", None) is None:
            import pdb; pdb.set_trace()
        print("Skip", _i)
        print("Debug: -----")
        for __each in _score_debug:
            print(__each)
            print("--")
        continue
    advantages = (meaning_scores - meaning_scores.mean()) / (meaning_scores.std() + 1e-5)
    advantages = advantages[...,None]
    advantages = advantages.to(inputs.device)

    # get the generated outputs only
    print(_i)
    for _mu_i in range(mu):
        optimizer.zero_grad()
        logits = peft_model(data_output).logits

        # only concern with the generated output up before the generation of the last token
        logits = logits[:, inputs.shape[1]-1:-1]
        
        # get the log-softmax
        logsumexp = torch.logsumexp(logits, dim=-1, keepdim=True)
        logprobs = logits - logsumexp

        # logps = logprobs[torch.arange(8)[...,None],torch.arange(43)[None,...],ground_truth]
        logps = torch.gather(logprobs, dim=-1, index=ground_truth[...,None]).squeeze() * mask
        if _mu_i == 0:
            logps_old = logps.detach()

        margin_logprobs = torch.exp(logps - logps_old)
        coeff1 = margin_logprobs * advantages
        coeff2 = torch.clip(margin_logprobs, 1 - epsilon, 1 + epsilon) * advantages
        loss = torch.minimum(coeff1, coeff2).sum(dim=-1)

        # calculate KL
        kl_log_diff = ref_logps - logps
        kl = torch.exp(kl_log_diff) - kl_log_diff - 1
        kl = kl.sum(dim=-1)

        loss -= beta * kl
        loss = -loss
        loss = loss.mean()
        print(f"{loss=}")
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    

--------------------------------------------------
SUBREDDIT: r/relationships

TITLE: I (f/22) have to figure out if I want to still know these girls or not and would hate to sound insulting

POST: Not sure if this belongs here but it's worth a try. 

Backstory:
When I (f/22) went through my first real breakup 2 years ago because he needed space after a year of dating roand  it effected me more than I thought. It was a horrible time in my life due to living with my mother and finally having the chance to cut her out of my life. I can admit because of it was an emotional wreck and this guy was stable and didn't know how to deal with me. We ended by him avoiding for a month or so after going to a festival with my friends. When I think back I wish he just ended. So after he ended it added my depression I suffered but my friends helped me through it and I got rid of everything from him along with cutting contact. 

Now: Its been almost 3 years now and I've gotten better after counselling a

ipdb>  c


Skip 44
Debug: -----
<think>
Okay, let's see. The user is asking for a good first motorcycle for a beginner, and there's a post in r/AskReddit. The original summary 1 is "What's a good first motorcycle for a beginner?" and the summary 2 is "new biker in SF bay-area needs advice on what kinds of standard/cruiser motorcycles are good for around-the-town driving."

First, I need to compare these two summaries. The first one is a general question asking about good motorcycles for beginners. The second one is more specific, mentioning the user's location (SF Bay Area) and the need for standard/cruiser bikes for city driving. 

So, summary 1 is more general. The user is in SF, so the second summary is tailored to their specific situation. The first summary doesn't mention their location, making it less relevant. Therefore, the first summary is closer to the second summary than the other way around. The user might have been trying to find a recommendation for a beginner in SF, which the secon

ipdb>  c


Skip 62
Debug: -----
<think>
Okay, let's see. The user wants to know how close the first summary is to the second summary from 0 to 5. 

First, I need to compare the two summaries. The first one is "r/tifu TIFU by Letting my Brothers onto an Social Experiment I was Planning." The second is "I attempted an incognito social experiment to estimate to see how many bronies live in my area and got it blown wide open by way of discussion of video games."

Looking at the first summary, it starts with the name of the brother, mentions the social experiment, and the name Fluttershy. The second summary talks about trying to estimate the number of bronies in their area through a game discussion. The key difference here is that the first summary mentions a social experiment involving a pup named Fluttershy, while the second talks about a game discussion. 

So the first summary is about the social experiment with the pup, the second is about estimating bronies using a game. Since the first summary d

ipdb>  c


Skip 117
Debug: -----
<think>
Okay, let me compare the two summaries. 

First, the summary 1 is "I'm a 'penetrating anal cake'! 🍑". The second is "found an eraser that smelled like a urinal cake. Freudian slipped it to anal cake for my English teacher."

So, the first one is a direct statement about being a cake that's penetrating, while the second mentions finding a smell and slipping it into the teacher's mouth. The second one adds more details about how the teacher didn't know the smell, which the brain responded to. The first summary doesn't include the teacher's reaction, only the initial reaction.

The first summary is more concise and just states the outcome, while the second one includes the process and the teacher's reaction. The original text also mentions the teacher's reaction and the brain's response. The first summary is more straightforward, while the second includes a bit more narrative.

So, the first summary is 4 out of 5, as the second one is more detailed and includ

ipdb>  exit


In [31]:
print(meaning_scores)

[5.0]


In [25]:
print(prompt)

SUBREDDIT: r/relationships

TITLE: I (27/F) am obnoxiously paranoid after a gynecological health scare. How do I explain my ridiculous hormonal-ness to my boyfriend (32/M)? (X-posted to /r/AskWomen)

POST: Last month, I started getting really hormonal and feeling downright crazy (becoming paranoid about my awesome boyfriend for absolutely no reason, crying at the drop of a hat, and all of that jazz.). Right around that time, my gyno told me that I was due to have my IUD removed. Oh, okay! That's why I was a hormonal wreck! Bada bing, bada boom, I'll get it replaced and everything will be back to normal.

When I got to his office for what was meant to be a routine appointment, he found a large mass in one of my ovaries--and told me that I needed to have a hysteroscopy to get my IUD removed. He scheduled the hysteroscopy and the aspiration of the mass for the next day. I was horrified and scared shitless. My boyfriend was very supportive, stayed up late rubbing my neck the night before t

In [27]:
print(_data["completion"])

 Had a big-ass ovarian cyst; scared that I'm irreparably broken; hormonal shitstorm ensues; worried I'm going to alienate my boyfriend with my behavior.


In [24]:
for _each in generated_candidates:
    print(_each)

r/relationships  
TL;DR: I'm paranoid and scared after a gynecological scare. I need to let my boyfriend know I'm not just crazy, but I'm being irrational.
r/relationships  
TL;DR: I'm paranoid and scared after a gynecological scare, but I'm trying to explain it to my guy.
r/relationships  
TL;DR: I'm paranoid and scared after a gynecological scare, but I'm trying to explain my hormonal issues to my boyfriend.


In [23]:
meaning_scores

tensor([0, 0, 0])

In [452]:
with torch.no_grad():
    kl_log_diff = ref_logps - logps
    kl = torch.exp(kl_log_diff) - kl_log_diff - 1
    print(kl)

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.8327e-04, 0.0000e+00,
         0.0000e+00, 1.9193e-05, 4.4191e-04, 2.0042e-02, 1.1715e-03, 1.3795e-03,
         1.9256e+01, 2.6671e+01, 5.9605e-07, 2.0112e-03, 1.8752e-04, 3.0833e+01,
         1.9973e-01, 4.6656e+00, 1.4111e+01, 9.7411e+00, 1.5058e-01, 1.8473e+01,
         1.1471e+00, 1.8209e-01, 5.0901e+00, 9.2383e-02, 5.1409e-01, 4.9937e+00,
         4.1446e-01, 5.0184e-01, 8.2251e-03, 6.5731e-01, 5.8401e+00, 5.0459e+00,
         1.6983e+00, 1.6273e+01, 8.0135e-03, 4.7755e-04],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.8327e-04, 0.0000e+00,
         0.0000e+00, 1.9193e-05, 4.4191e-04, 2.0042e-02, 1.1715e-03, 1.3795e-03,
         1.1952e-01, 3.5405e-02, 2.3842e-06, 5.5628e-03, 9.2983e-05, 1.9405e-01,
         1.7269e-01, 5.7495e-03, 6.3694e-02, 9.9773e-02, 2.7379e-01, 8.8215e-06,
         2.1005e-03, 8.4565e-01, 2.7204e-01, 2.4058e-02, 8.6330e-01, 2.1248e-02,
         6.6757e-06, 8.8693e-03, 0.0000e+00, 0.0000

In [412]:
print(f"{advantages.shape=}")
print(f"{logps.shape=}")

with torch.no_grad():
    coeff1 = logps * advantages
print(f"{coeff1.shape=}")

advantages.shape=torch.Size([8, 1])
logps.shape=torch.Size([8, 37])
coeff1.shape=torch.Size([8, 37])


In [20]:
len(trained_params), len(original_params)

(6, 6)

In [26]:
for __idx, __each in enumerate(trained_params):
    print(__idx)
    print("Trained:", __idx, __each.norm(), __each.shape)
    print("Original:", original_params[__idx].norm(), original_params[__idx].shape)

0
Trained: 0 tensor(2.4932, device='mps:0') torch.Size([16, 1024])
Original: tensor(2.3169, device='mps:0') torch.Size([16, 1024])
1
Trained: 1 tensor(1.0636, device='mps:0') torch.Size([3072, 16])
Original: tensor(0., device='mps:0') torch.Size([3072, 16])
2
Trained: 2 tensor(2.4710, device='mps:0') torch.Size([16, 1024])
Original: tensor(2.3368, device='mps:0') torch.Size([16, 1024])
3
Trained: 3 tensor(1.0300, device='mps:0') torch.Size([3072, 16])
Original: tensor(0., device='mps:0') torch.Size([3072, 16])
4
Trained: 4 tensor(2.6436, device='mps:0') torch.Size([16, 3072])
Original: tensor(2.3065, device='mps:0') torch.Size([16, 3072])
5
Trained: 5 tensor(0.6417, device='mps:0') torch.Size([1024, 16])
Original: tensor(0., device='mps:0') torch.Size([1024, 16])


In [34]:
for _temp in range(6):
    print(_temp)
    print((trained_params[_temp] - original_params[_temp]).norm() / torch.maximum(original_params[_temp].norm(), trained_params[_temp].norm()))

0
tensor(0.3468, device='mps:0')
1
tensor(1., device='mps:0')
2
tensor(0.3000, device='mps:0')
3
tensor(1., device='mps:0')
4
tensor(0.4793, device='mps:0')
5
tensor(1., device='mps:0')


In [23]:
with torch.no_grad():
    for _n, _p in peft_model.named_parameters():
        if _p.requires_grad:
            print(_n)

base_model.model.model.layers.26.mlp.gate_proj.lora_A.default.weight
base_model.model.model.layers.26.mlp.gate_proj.lora_B.default.weight
base_model.model.model.layers.26.mlp.up_proj.lora_A.default.weight
base_model.model.model.layers.26.mlp.up_proj.lora_B.default.weight
base_model.model.model.layers.26.mlp.down_proj.lora_A.default.weight
base_model.model.model.layers.26.mlp.down_proj.lora_B.default.weight


In [39]:
print(peft_model.base_model.model.model.layers[26].mlp.gate_proj.lora_A['default'])
print(peft_model.base_model.model.model.layers[26].mlp.gate_proj.lora_B['default'])

Linear(in_features=1024, out_features=16, bias=False)
Linear(in_features=16, out_features=3072, bias=False)


In [40]:
delta_weights = peft_model.base_model.model.model.layers[26].mlp.gate_proj.get_delta_weight("default")

In [41]:
delta_weights.shape

torch.Size([3072, 1024])

In [45]:
peft_model.base_model.model.model.layers[26].mlp.gate_proj.base_layer.weight.shape

torch.Size([3072, 1024])

In [54]:
delta_weights = peft_model.base_model.model.model.layers[26].mlp.gate_proj.get_delta_weight("default")
gate_proj_after = peft_model.base_model.model.model.layers[26].mlp.gate_proj.base_layer.weight.data + delta_weights * 2.0
gate_proj_before = torch.clone(peft_model.base_model.model.model.layers[26].mlp.gate_proj.base_layer.weight.data)

In [119]:
_layer = peft_model.base_model.model.model.layers[26].mlp.gate_proj
_delta = _layer.get_delta_weight("default")
_after = _layer.base_layer.weight.data + _delta * _layer.scaling["default"]
_before = torch.clone(_layer.base_layer.weight.data)

_ = compare_weight_by_svd(_before, _after)

-- Comparing e1, e2:
linalg.cosine_similarity(e1,e2)=0.99995106
np.linalg.norm(e1)=49.623543
np.linalg.norm(e2)=49.774715
norm_diff=0.51322764
diff_pct=0.0103
-- Comparing u1, u2:
subspace angles: [6.9641928e-06 6.8092763e-06 6.6213011e-06 ... 2.8858362e-09 1.2190413e-09
 6.6457240e-10]
max angle: 6.964192834857386e-06
mean angle: 2.1373268737079343e-06
min angle: 6.645723971132611e-10
-- Comparing v1, v2:
subspace angles: [3.5123830e-06 3.4531702e-06 3.3566475e-06 ... 5.1967275e-09 1.5006500e-09
 1.1752321e-09]
max angle: 3.512383045745082e-06
mean angle: 1.1330463394187973e-06
min angle: 1.1752321338320826e-09
linalg.is_orthonormal(Q_estimate)=True
linalg.is_orthonormal(R_estimate)=True
0.10615542


In [120]:
_layer = peft_model.base_model.model.model.layers[26].mlp.up_proj
_delta = _layer.get_delta_weight("default")
_after = _layer.base_layer.weight.data + _delta * _layer.scaling["default"]
_before = torch.clone(_layer.base_layer.weight.data)

_ = compare_weight_by_svd(_before, _after)

-- Comparing e1, e2:
linalg.cosine_similarity(e1,e2)=0.99996907
np.linalg.norm(e1)=60.62313
np.linalg.norm(e2)=60.723568
norm_diff=0.48722282
diff_pct=0.0080
-- Comparing u1, u2:
subspace angles: [7.7407522e-06 7.5101834e-06 7.2304210e-06 ... 2.1036244e-09 1.5370317e-09
 1.3066408e-10]
max angle: 7.740752153040376e-06
mean angle: 2.1682865281036356e-06
min angle: 1.3066407678063996e-10
-- Comparing v1, v2:
subspace angles: [3.3508425e-06 3.2956343e-06 3.2799303e-06 ... 3.3233329e-09 4.3687665e-10
 2.8407288e-10]
max angle: 3.3508424621686572e-06
mean angle: 1.1196659670531517e-06
min angle: 2.8407287633314127e-10
linalg.is_orthonormal(Q_estimate)=True
linalg.is_orthonormal(R_estimate)=True
0.07756354


In [121]:
_layer = peft_model.base_model.model.model.layers[26].mlp.down_proj
_delta = _layer.get_delta_weight("default")
_after = _layer.base_layer.weight.data + _delta * _layer.scaling["default"]
_before = torch.clone(_layer.base_layer.weight.data)

_ = compare_weight_by_svd(_before, _after)

-- Comparing e1, e2:
linalg.cosine_similarity(e1,e2)=0.99998647
np.linalg.norm(e1)=56.25277
np.linalg.norm(e2)=56.34758
norm_diff=0.30850995
diff_pct=0.0055
-- Comparing u1, u2:
subspace angles: [3.9200240e-06 3.4942527e-06 3.4749514e-06 ... 5.2788809e-09 3.1037117e-09
 5.5605687e-10]
max angle: 3.920024028047919e-06
mean angle: 1.159133489636588e-06
min angle: 5.560568672180466e-10
-- Comparing v1, v2:
subspace angles: [7.4262084e-06 7.2382941e-06 7.0757615e-06 ... 3.5062309e-09 1.2483174e-09
 1.5459088e-10]
max angle: 7.426208412653068e-06
mean angle: 2.168339960917365e-06
min angle: 1.5459088176239533e-10
linalg.is_orthonormal(Q_estimate)=True
linalg.is_orthonormal(R_estimate)=True
0.07907779


In [116]:
peft_model.base_model.model.model.layers[26].mlp.up_proj.scaling

{'default': 2.0, 'ref_model': 2.0}

In [115]:
print(peft_model.base_model.model.model.layers[26].mlp)

Qwen3MLP(
  (gate_proj): lora.Linear(
    (base_layer): Linear(in_features=1024, out_features=3072, bias=False)
    (lora_dropout): ModuleDict(
      (default): Identity()
      (ref_model): Identity()
    )
    (lora_A): ModuleDict(
      (default): Linear(in_features=1024, out_features=16, bias=False)
      (ref_model): Linear(in_features=1024, out_features=16, bias=False)
    )
    (lora_B): ModuleDict(
      (default): Linear(in_features=16, out_features=3072, bias=False)
      (ref_model): Linear(in_features=16, out_features=3072, bias=False)
    )
    (lora_embedding_A): ParameterDict()
    (lora_embedding_B): ParameterDict()
    (lora_magnitude_vector): ModuleDict()
  )
  (up_proj): lora.Linear(
    (base_layer): Linear(in_features=1024, out_features=3072, bias=False)
    (lora_dropout): ModuleDict(
      (default): Identity()
      (ref_model): Identity()
    )
    (lora_A): ModuleDict(
      (default): Linear(in_features=1024, out_features=16, bias=False)
      (ref_model): Li

In [48]:
gate_proj_before = torch.clone(peft_model.base_model.model.model.layers[26].mlp.gate_proj.base_layer.weight.data)

In [114]:
@torch.no_grad()
def compare_weight_by_svd(w1, w2, verbose=True):
  if isinstance(w1, torch.Tensor):
    w1 = w1.cpu().numpy()
  if isinstance(w2, torch.Tensor):
    w2 = w2.cpu().numpy()

  if len(w1.shape) != 2:
    raise ValueError(f"Expected 2D tensor, got {w1.shape}")
  if len(w2.shape) != 2:
    raise ValueError(f"Expected 2D tensor, got {w2.shape}")

  if w1.shape != w2.shape:
    raise ValueError(f"Expected same shape, got {w1.shape} and {w2.shape}")

  u1, e1, v1 = np.linalg.svd(w1)
  u2, e2, v2 = np.linalg.svd(w2)

  norm_diff = linalg.norm(e1 - e2)
  diff_pct = norm_diff / max(linalg.norm(e1), linalg.norm(e2))

  if verbose:
    print("-- Comparing e1, e2:")
    print(f"{linalg.cosine_similarity(e1,e2)=}")
    print(f"{np.linalg.norm(e1)=}")
    print(f"{np.linalg.norm(e2)=}")
    print(f"{norm_diff=}")
    print(f"{diff_pct=:.4f}")

  u_angles = subspace_angles(u1, u2)
  if verbose:
    print("-- Comparing u1, u2:")
    print(f"subspace angles: {u_angles}")
    print(f"max angle: {u_angles.max()}")
    print(f"mean angle: {u_angles.mean()}")  
    print(f"min angle: {u_angles.min()}")

  v_angles = subspace_angles(v1, v2)
  if verbose:
    print("-- Comparing v1, v2:")
    print(f"subspace angles: {v_angles}")
    print(f"max angle: {v_angles.max()}")
    print(f"mean angle: {v_angles.mean()}")  
    print(f"min angle: {v_angles.min()}")

  Q_estimate = u2 @ u1.T
  R_estimate = v1.T @ v2

  if verbose:
    print(f"{linalg.is_orthonormal(Q_estimate)=}")
    print(f"{linalg.is_orthonormal(R_estimate)=}")

  w2r = Q_estimate @ w2 @ R_estimate
  print(abs(np.linalg.norm(w2r - w1) / np.linalg.norm(w1)))
  return ((u1, e1, v1), (u2, e2, v2))


((u1, e1, v1), (u2, e2, v2)) = compare_weight_by_svd(gate_proj_before, gate_proj_after)

-- Comparing e1, e2:
linalg.cosine_similarity(e1,e2)=0.99995106
np.linalg.norm(e1)=49.623543
np.linalg.norm(e2)=49.774715
norm_diff=0.51322764
diff_pct=0.0103
-- Comparing u1, u2:
subspace angles: [6.9641928e-06 6.8092763e-06 6.6213011e-06 ... 2.8858362e-09 1.2190413e-09
 6.6457240e-10]
max angle: 6.964192834857386e-06
mean angle: 2.1373268737079343e-06
min angle: 6.645723971132611e-10
-- Comparing v1, v2:
subspace angles: [3.5123830e-06 3.4531702e-06 3.3566475e-06 ... 5.1967275e-09 1.5006500e-09
 1.1752321e-09]
max angle: 3.512383045745082e-06
mean angle: 1.1330463394187973e-06
min angle: 1.1752321338320826e-09
linalg.is_orthonormal(Q_estimate)=True
linalg.is_orthonormal(R_estimate)=True
0.10615542


In [113]:
subspace_angles(u1, u2)

array([6.9641928e-06, 6.8092763e-06, 6.6213011e-06, ..., 2.8858362e-09,
       1.2190413e-09, 6.6457240e-10], dtype=float32)

In [58]:
gate_proj_after.shape

torch.Size([3072, 1024])

In [55]:
(gate_proj_after - gate_proj_before).norm() / gate_proj_before.norm()

tensor(0.0780, device='mps:0')

In [56]:
(gate_proj_after - gate_proj_before).norm()

tensor(3.8725, device='mps:0')

In [57]:
gate_proj_before.norm()

tensor(49.6235, device='mps:0')

In [52]:
gate_proj_after.norm()

tensor(99.3229, device='mps:0')

In [343]:
peft_model.active_adapters

['default']

In [229]:
data_output = model.generate(inputs, max_new_tokens=100)

In [243]:
data_output.shape

torch.Size([8, 490])

In [259]:
print(f"{logprobs.shape=}")
print(f"{ground_truth.shape=}")

logprobs.shape=torch.Size([8, 43, 151936])
ground_truth.shape=torch.Size([8, 43])


In [275]:
_temp_out = logprobs[torch.arange(8)[...,None],torch.arange(43)[None,...],ground_truth]

In [312]:
_temp_out2 = torch.gather(logprobs, dim=-1, index=ground_truth[...,None])

In [313]:
_temp_out2.shape

torch.Size([8, 43, 1])

In [315]:
torch.allclose(_temp_out, _temp_out2.squeeze())

True

In [288]:
_idx0, _idx1 = 5, 10
print(_idx0, _idx1, ground_truth[_idx0,_idx1])
print(_temp_out[_idx0, _idx1])
print(logprobs[_idx0, _idx1, ground_truth[_idx0,_idx1]])

logprobs[5,10,69:80]

5 10 tensor(69, device='mps:0')
tensor(-0.0039, device='mps:0', grad_fn=<SelectBackward0>)
tensor(-0.0039, device='mps:0', grad_fn=<SelectBackward0>)


tensor([-3.8891e-03, -1.2047e+01, -1.2966e+01, -1.1012e+01, -1.4238e+01,
        -1.2882e+01, -1.2093e+01, -1.1383e+01, -1.3150e+01, -1.3351e+01,
        -1.4089e+01], device='mps:0', grad_fn=<SliceBackward0>)

In [295]:
logits[5,10,69:80]

tensor([24.7626, 12.7194, 11.8005, 13.7549, 10.5286, 11.8849, 12.6735, 13.3834,
        11.6169, 11.4151, 10.6776], device='mps:0', grad_fn=<SliceBackward0>)

In [297]:
logsumexp[5,10]

tensor([24.7665], device='mps:0', grad_fn=<SelectBackward0>)

In [302]:
exp = torch.exp(logits[5,10])
prob = exp / exp.sum()

In [304]:
prob[69:80]

tensor([9.9612e-01, 5.8618e-06, 2.3386e-06, 1.6509e-05, 6.5550e-07, 2.5446e-06,
        5.5989e-06, 1.1387e-05, 1.9463e-06, 1.5907e-06, 7.6085e-07],
       device='mps:0', grad_fn=<SliceBackward0>)

In [305]:
torch.exp(logprobs[5,10,69:80])

tensor([9.9612e-01, 5.8618e-06, 2.3386e-06, 1.6509e-05, 6.5551e-07, 2.5446e-06,
        5.5989e-06, 1.1387e-05, 1.9463e-06, 1.5907e-06, 7.6085e-07],
       device='mps:0', grad_fn=<ExpBackward0>)

In [269]:
_idx = torch.tensor([0,1,2,1,0])

In [272]:
_x[torch.arange(5),_idx]

tensor([0.7626, 0.7875, 0.3994, 0.3482, 0.9230])

In [ ]:
for name, param in model.named_parameters():
    if not param.requires_grad:
        continue
    with torch.no_grad():
        print(name, param.sum())
        print(param)

In [244]:
inputs.shape

torch.Size([8, 447])

In [231]:
with torch.no_grad():
    data_output2 = model(data_output)

In [233]:
data_output2.logits.shape

torch.Size([8, 490, 151936])

In [234]:
data_output.shape

torch.Size([8, 490])

In [222]:
data_output['sequences'].shape

torch.Size([8, 495])

In [223]:
logits = data_output["logits"]

In [210]:
print(data_output)

tensor([[151644,    872,    198,  ..., 151643, 151643, 151643],
        [151644,    872,    198,  ...,  11303,      0, 151645],
        [151644,    872,    198,  ..., 151643, 151643, 151643],
        ...,
        [151644,    872,    198,  ..., 151643, 151643, 151643],
        [151644,    872,    198,  ..., 151643, 151643, 151643],
        [151644,    872,    198,  ..., 151643, 151643, 151643]],
       device='mps:0')


In [208]:
data_output.shape

torch.Size([8, 504])

In [209]:
for name, params in model.named_parameters():
    if not params.requires_grad:
        continue
    print(name, params.grad)

model.layers.26.mlp.gate_proj.lora_A.default.weight None
model.layers.26.mlp.gate_proj.lora_B.default.weight None
model.layers.26.mlp.up_proj.lora_A.default.weight None
model.layers.26.mlp.up_proj.lora_B.default.weight None
model.layers.26.mlp.down_proj.lora_A.default.weight None
model.layers.26.mlp.down_proj.lora_B.default.weight None


In [4]:
trainer = GRPOTrainer(
    model="Qwen/Qwen3-0.6B",
    reward_funcs=reward_num_unique_chars,
    train_dataset=dataset,
    peft_config=peft_config,
)

In [6]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
wandb: Currently logged in as: trungducdeptrai-loitaiai (trungducdeptrai-loitaiai-ai-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/Users/john/venv/ctm/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`generation_config` default values have been modified to match model-specific defaults: {'temperature': 0.6, 'top_p': 0.95}. If this is not desired, please set these values explicitly.


Step,Training Loss
10,-0.000000
20,0.000000
30,0.000000
40,0.000000
50,-0.000000
60,0.000000
70,0.000000
80,0.000000
90,0.000000
100,-0.000000


KeyboardInterrupt: 